Noteneingabetool für HISinOne
=============================
Dieses Jupyter-Notebook ist ein interaktives Tool zur Noteneingabe in HISinOne. Es können exportierte HISInOne Exceltabellen geladen werden, woraus eine interaktive Tabelle zum Eingeben der Punkte in einzelnen Aufgaben erzeugt wird. Aus der Summe der Punkte können Noten berechnet werden sowie Statistken wie z.B. die Notenverteilung dargstellt werden.

Die Noten können dann wieder im HISinOne Format exportiert werden und in HISinOne hochgeladen werden.

---
Konfiguration:
--------------

In [19]:
# Die zu importierende HISInOne-Tabelle (über die Funktion Excel-Export der Noteneingabe in HISInOne erzeugt)
table_name = '5160-RocketScience-WiSe_2023.xlsx' 
# Namen der Aufgaben, für die Punkte eintragen werden sollen
questions = ['1', '2', '3', '4', '5a', '5b', '6']
# Parameter für Noteberechnung
max_points = 40     # Maximale Punktzahl
grade_1_0 = 0.92    # Prozentuale Grenze für Note 1,0
grade_4_0 = 0.4     # Prozentuale Grenze für Note 4,0

---
Namensimport
------------

In [20]:
import pandas as pd
import numpy as np

st = pd.read_excel(table_name, header=None)

# Finde Indizes der Zeilen mit den Werten "startHISsheet" und "endHISsheet"
def find_row_with_value(df, value):
    mask = st.iloc[:, 0] == value
    if not mask.any():
        raise ValueError(f'{value} nicht gefunden')    
    return mask.idxmax()

start_idx = find_row_with_value(st, 'startHISsheet')
end_idx = find_row_with_value(st, 'endHISsheet')
#print(start_idx,end_idx)
# Lade Daten im korrekten Bereich
st = pd.read_excel(table_name,
                   header=start_idx+1,
                   nrows=end_idx-start_idx-2,
                   index_col='Matrikelnummer',
                   dtype={'Leistung': str})

print('Folgende Tabelle wurde importiert:')
st

3 11
Folgende Tabelle wurde importiert:


,Examplan.id,PrüfungsNr.,Titel,Nachname,Vorname,Leistung,Versuch,Status,ECTS,Semester,Jahr,Prüfungsperiode,Vermerk,Thema,Beginn,Gepl. Ende,Tatsächl. Ende,Prüfungsart,Prüfungsform,LockVersion
Matrikelnummer,,,,,,,,,,,,,,,,,,,,
3434343,1414141,5160,Informatik 2,Hendrix,Jimi,NaN,1,zugelassen,NaN,Wintersemester,2023,1,NaN,NaN,NaN,NaN,NaN,Prüfung,Schriftlich,"ea56ef45-1b42-4a9f-8dc2-e9f3b8e6c9a1=0,109aefb..."
3535353,1515151,5160,Informatik 2,Lukather,Steve,NaN,1,zugelassen,NaN,Wintersemester,2023,1,NaN,NaN,NaN,NaN,NaN,Prüfung,Schriftlich,"1e851fa6-6f69-4fe4-a1a8-510ab8e5c6ea=0,eeecd36..."
3131313,1111111,5160,Informatik 2,Mayer,John,NaN,3,zugelassen,NaN,Wintersemester,2023,1,NaN,NaN,NaN,NaN,NaN,Prüfung,Schriftlich,"fd952c3a-c789-4af2-88fd-0dbe8ea0e68b=0,5a639ec..."
2323232,1313131,5160,Informatik 2,Page,Jimmy,NaN,2,zugelassen,NaN,Wintersemester,2023,1,NaN,NaN,NaN,NaN,NaN,Prüfung,Schriftlich,"ee3f3068-dad7-489c-a5e7-f2eccaedacf7=0,92e970b..."
3737373,1717171,5160,Informatik 2,Townshend,Pete,NaN,1,zugelassen,NaN,Wintersemester,2023,1,NaN,NaN,NaN,NaN,NaN,Prüfung,Schriftlich,"1832c736-1d8c-4d49-bdcd-4295fc5bf3b4=0,8f5bf01..."
3939393,1919191,5160,Informatik 2,Van Halen,Eddie,NaN,1,zugelassen,NaN,Wintersemester,2023,1,NaN,NaN,NaN,NaN,NaN,Prüfung,Schriftlich,"14d1b781-a879-4115-a444-c87b725bbfe2=0,98e734b..."


---
Punkteeingabe
-------------

In [21]:
from pathlib import Path
from ipydatagrid import DataGrid

# Erzeuge Tabelle zur Punkteeingabe
select_columns = st[['Nachname', 'Vorname']]
new_col = pd.Series([np.nan] * len(st), name='1', index=st.index)
st_points = pd.concat([select_columns, new_col], axis=1)
add_columns = [pd.Series([np.nan] * len(st), name=q, index=st.index) for q in questions]
st_points = pd.concat([select_columns] + add_columns, axis=1)

# Speichere Tabelle zur Punkteeingabe bzw. lade sie, falls sie bereits existiert
storage_path = Path() / Path(table_name + '_editable_df.pkl')

# ACHTUNG: Folgende Zeile auskommentieren zum Zurücksetzen der Änderungen! Dies ist irreversibel!
#storage_path.unlink()

if storage_path.exists():
    print(f'Stelle Änderungen aus {storage_path} wieder her.')
    df = pd.read_pickle(storage_path)
else:
    print(f'Keine Änderungen gefunden, werde zukünftige Änderungen nach {storage_path} speichern.')
    df = st_points
    
datagrid = DataGrid(df, editable=True)
datagrid.on_cell_change(lambda cell: datagrid.data.to_pickle(storage_path))

datagrid

Keine Änderungen gefunden, werde zukünftige Änderungen nach 5160-RocketScience-WiSe_2023.xlsx_editable_df.pkl speichern.


DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_renderer=None, default_render…

In [22]:
import numpy as np

grades = ['1,0', '1,3', '1,7', '2,0', '2,3', '2,7', '3,0', '3,3', '3,7', '4,0', '5,0']

grade_interval = (grade_1_0 - grade_4_0) / 9

grade_intervals = [grade_1_0 - i * grade_interval for i in range(10)]
grade_intervals.append(0)
points = np.array([i * max_points for i in grade_intervals])

score_table = pd.DataFrame({'Note': grades, 'Prozent': grade_intervals, 'Untergrenze': points})

print(f'Maximalpunktzahl: {max_points}')
print(f'1.0-Grenze (%): {grade_1_0}')
print(f'4.0-Grenze (%): {grade_4_0}')
print('-----')
print('Notentabelle:')
score_table

Maximalpunktzahl: 40
1.0-Grenze (%): 0.92
4.0-Grenze (%): 0.4
-----
Notentabelle:


,Note,Prozent,Untergrenze
0,"1,0",0.920000,36.800000
1,"1,3",0.862222,34.488889
2,"1,7",0.804444,32.177778
3,"2,0",0.746667,29.866667
4,"2,3",0.688889,27.555556
5,"2,7",0.631111,25.244444
6,"3,0",0.573333,22.933333
7,"3,3",0.515556,20.622222
8,"3,7",0.457778,18.311111
9,"4,0",0.400000,16.000000


In [23]:
# Berechne Summe und Note
df = datagrid.data
df['Summe'] = df[questions].sum(axis=1, min_count=len(questions))

score_table.iloc[index]['Note']
df['Note'] = df['Summe'].apply(lambda x: np.nan if np.isnan(x) else score_table.iloc[np.digitize(x, points)]['Note'])

print('Folgende Punktesummen und Noten wurden berechnet:')
df

Folgende Punktesummen und Noten wurden berechnet:


,Nachname,Vorname,1,2,3,4,5a,5b,6,Summe,Note
Matrikelnummer,,,,,,,,,,,
3434343,Hendrix,Jimi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3535353,Lukather,Steve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3131313,Mayer,John,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2323232,Page,Jimmy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3737373,Townshend,Pete,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3939393,Van Halen,Eddie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
Notenverteilung
---------------

In [24]:
pd.options.plotting.backend = "plotly"

score_hist = pd.DataFrame({'Anzahl': [0] * len(grades)}, index=score_table['Note'])

counts = pd.DataFrame(df['Note'].value_counts().rename('Anzahl'))
score_hist.update(counts)

print('Notenverteilung:')
print(score_hist)

number_students = len(df)
number_participants = len(df[df['Summe'].notna()])
print(f'Anzahl Anmeldungen: {number_students}')
print(f'Anzahl Teilnehmer: {number_participants}')
score_hist.plot(kind='bar')

Notenverteilung:
      Anzahl
Note        
1,0        0
1,3        0
1,7        0
2,0        0
2,3        0
2,7        0
3,0        0
3,3        0
3,7        0
4,0        0
5,0        0
Anzahl Anmeldungen: 6
Anzahl Teilnehmer: 0


---
Export
---------------

In [26]:
st['Leistung'] = df['Note'].replace({np.nan: 'NT'})
st_out = st.set_index('Examplan.id')

export_table = pd.read_excel(table_name, header=None, index_col=0)
export_table[[6]] = st_out[['Leistung']]

export_table.to_excel(table_name + '_Noten.xlsx', sheet_name='First Sheet', header=False, index=True)